In [21]:
offset_list = []
with open('/Users/yao/Desktop/band offset/offset_list.csv', 'r') as f:
    for line in f:
        offset_list.append([line.split(',')[0], line.split(',')[1], float(line.split(',')[2])])
offset_dict = {}
for i in offset_list:
    offset_dict[i[0]+'/'+i[1]] = i[2]

In [2]:
def get_all_compounds(csvlist):
    """return a set of all compounds"""
    comp_list = []
    for i in range(1, len(csvlist)):
        csvlist[i][0] = csvlist[i][0].replace(" ", "")
        compoundA = csvlist[i][0].split('/')[0]
        compoundB = csvlist[i][0].split('/')[1]
        comp_list.append(compoundA)
        comp_list.append(compoundB)
    return set(comp_list)

In [3]:
import csv
import numpy as np
import re
import os
with open('/Users/yao/Desktop/band offset/band_offset_collect.csv', 'r') as f:
    reader = csv.reader(f)
    csvlist = list(reader)
    comp_set = get_all_compounds(csvlist)
    del_set = set(['CH3NH3PbBr3', 'CH3NH3PbCl3', 'CH3NH3PbI3', 'CH3NH3PbIBr2', 'HC(NH2)2PbI3',
               'Er2O3', 'Fe2O3', 'Gd2O3', 'HfO2', 'InGaZnO4', 'CuInGaSe2', 'InSnO', 'Lu2O3',
               'NiO', 'NiOx', 'Pr2O3', 'Tm2O3'])
    desired_set = comp_set-del_set
    desired_list = [i for i in desired_set]

In [4]:
graph = {}
for i in offset_list:
    compA = i[0]
    compB = i[1]
    if (compA in desired_list) and (compB in desired_list):
        if compA not in graph:
            graph[compA] = set([compB])
        else:
            graph[compA].add(compB)

In [5]:
def dfs_paths(graph, start, goal):
    stack = [(start, [start])]
    while stack:
        (vertex, path) = stack.pop()
        for next in graph[vertex] - set(path):
            if next == goal:
                yield path + [next]
            else:
                stack.append((next, path + [next]))

In [6]:
def bfs_paths(graph, start, goal):
    queue = [(start, [start])]
    while queue:
        (vertex, path) = queue.pop(0)
        for next in graph[vertex] - set(path):
            if next == goal:
                yield path + [next]
            else:
                queue.append((next, path + [next]))
def shortest_path(graph, start, goal):
    try:
        return next(bfs_paths(graph, start, goal))
    except StopIteration:
        return None

In [22]:
graph_comp = [i for i in graph]
path_list = []
for i in range(len(graph_comp)):
    for j in range(i+1, len(graph_comp)):
        compA = graph_comp[i]
        compB = graph_comp[j]
        path = shortest_path(graph, compA , compB)
        path_list.append(path)

In [25]:
expanded_offset_dict = {}
for path in path_list:
    compA = path[0]
    compB = path[-1]
    offset = 0
    for i in range(len(path)-1):
        comp1 = path[i]
        comp2 = path[i+1]
        key = comp1+'/'+comp2
        offset += offset_dict[key]
    expanded_offset_dict[compA + '/' + compB] = offset

In [28]:
with open('/Users/yao/Desktop/band offset/expanded_offset.csv','w') as f:
    for i in expanded_offset_dict:
        f.write(i)
        f.write(',')
        f.write(str(expanded_offset_dict[i]))
        f.write('\n')